In [1]:
import matplotlib.pyplot as plt
import nibabel as nib
import os
import pandas as pd
import pickle
import scipy.ndimage as ndi
import warnings

from constants import Constants

from IPython.display import Image
from nilearn import plotting
from pathlib import Path

%matplotlib inline
warnings.filterwarnings('ignore')

In [3]:
constants = Constants()
unprocessed_data_path = constants.dataset_path / 'unprocessed'

unprocessed_data_path

WindowsPath('data/unprocessed')